In [6]:
pip install langchain_experimental

  Using cached langchain_experimental-0.3.0-py3-none-any.whl.metadata (1.7 kB)
Using cached langchain_experimental-0.3.0-py3-none-any.whl (206 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\venki\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
pip install pymysql

   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.0 kB ? eta -:--:--
   --------------------------- ------------ 30.7/45.0 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 369.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\venki\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from langchain_google_genai import GoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv() 

#llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])

C:\Users\venki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(llm("write few lines on langchain and Generative AI"))

C:\Users\venki\AppData\Local\Temp\ipykernel_28388\2314878354.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  print(llm("write few lines on langchain and Generative AI"))


LangChain is a powerful framework that bridges the gap between language models and real-world data. It empowers developers to build applications that seamlessly combine the power of Generative AI with external information sources, databases, and APIs. This results in more comprehensive and intelligent applications that can access and process real-world data, making them truly context-aware. 



#### Connect with database and ask some basic questions

In [7]:

from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

from langchain.utilities import SQLDatabase


In [10]:
db_user = "root"
db_password = "123456"
db_host = "localhost"
db_name = "retail_sales_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [41]:
#Convert question to SQL query
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many unique customers are there for each product category"})
response

'Question: How many unique customers are there for each product category\nSQLQuery: SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `Unique Customers` FROM `sales_tb` GROUP BY `ProductCategory`'

In [42]:

# Extracting the SQL query part from the response
cleaned_query = response.split('SQLQuery: ')[1].strip()
print(cleaned_query)


SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `Unique Customers` FROM `sales_tb` GROUP BY `ProductCategory`


In [43]:
# Execute the cleaned query
result = db.run(cleaned_query)
print(result)

[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [44]:
def execute_query(question, chain, db):
    # Generate the SQL query based on the question
    response = chain.invoke({"question": question})
    
    # Extract the SQL query part from the response
    cleaned_query = response.split('SQLQuery: ')[1].strip()
    
    # Execute the cleaned query
    result = db.run(cleaned_query)
    
    return result



In [49]:
question0 = "How many unique customers are there for each product category"
result = execute_query(question0, chain, db)

# Print the result
print(result)

[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [55]:
question = "How many unique customers are there for each product category"
result=execute_query(question, chain, db)

print(result)

[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [48]:
question2 = "Calculate total sales amount per product category:"
execute_query(question2, chain, db)

"[('Beauty', Decimal('1455.00')), ('Clothing', Decimal('5040.00')), ('Electronics', Decimal('5310.00'))]"

In [50]:
question3 = "calculates the average age of customers grouped by gender."
execute_query(question3, chain, db)

"[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]"

In [51]:
question4 = "identify the top spending customers based on their total amount spent."
execute_query(question4, chain, db)

"[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]"

In [53]:
question5 = "counts the number of transactions made each month."
execute_query(question5, chain, db)

'[(datetime.date(2023, 1, 13), 1), (datetime.date(2023, 1, 14), 1), (datetime.date(2023, 1, 16), 1), (datetime.date(2023, 1, 17), 1), (datetime.date(2023, 2, 14), 1)]'

In [54]:
question6 = "calculates the total sales amount and average price per unit for each product category."
execute_query(question6, chain, db)

"[('Beauty', Decimal('1455.00'), Decimal('153.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Electronics', Decimal('5310.00'), Decimal('238.750000'))]"